<div class="alert alert-block alert-success">
    <b>LAIMBIO</b>  Biomedical Engineering
</div>

## FSL commands - Automation
---

<div class="alert alert-block alert-info"> Pablo Laso Mielgo </div> 

<div class="alert alert-block alert-danger">
<b>Disclaimer:</b> <p> All folders, sub-folders and files within "main" directory must be described according to BIDS format.
    <br> This script is to be run in the path specified by "main".<p> Code failure will certainly ensue from not strictly fulfilling all thses specifications.
</div>

---


In [57]:
########################################################################################
##     author: Lasito                                                                ##
##     date: Feb, 18th                                                                ##
########################################################################################
## v2 is based in funtions                                                            ##
## last edition: fmap dimesions are equaled to dwi dimesions --> Mask step works.     ##
########################################################################################

import os
sep=os.path.sep
import json
from datetime import datetime

######################################################################################
main= sep+'Users'+sep+'administrator'+sep+'Documents'+sep+'niftii'       #############
######################################################################################
#main= r'C:\Users\pablo\OneDrive - Universidad Rey Juan Carlos\Biomedical Engineering URJC\Prácticas+Proyectos+Empresas+Contactos\Mario Gil Correa - Practicas-Biomedica\Documents\SENECA-PICASSO\niftii'

def check_dir(main):
    cwd = os.getcwd()
    if cwd != main:
        print('WARNING!!!!!')
    print('You are looking in:',main,'\nYour code is in   :', cwd)

def print_time():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return (current_time)

def json_params(files,save_dir,name,ckpt=0):
    texts=[]
    print("> creating params",print_time(),"      ", end="", flush=True)
    for file in files:
        with open(file) as f:
            d = json.load(f)
            AT=(d['AcquisitionMatrixPE'])
            ES=d['EffectiveEchoSpacing']
            j_direction=d['PhaseEncodingDirection']
            ReadOut_Time =AT*ES*0.5
            text= j_direction.replace('j-','0 -1 0 ').replace('j','0 1 0 ').replace('i-','-1 0 0 ').replace('i','1 0 0 ').replace('k-','0 0 -1 ').replace('k','0 0 1')+ str(ReadOut_Time)
            texts.append(text)
        with open(os.path.join(save_dir, name), 'w') as f:
            for item in texts:
                f.write("%s\n" % item)
    if ckpt:
        print('file saved as: ', save_dir+sep+name)
        print('content saved: ', texts)

def create_index(file,save_dir,name):
    print("> creating index",print_time(),"      ", end="", flush=True)
    with open(file) as f:
        lasito=f.read()
        volumes= len(lasito.split(' '))
    with open(os.path.join(save_dir, name), 'w') as f:
        for i in range(volumes):
            f.write("1 ")
            
def tidy_up(): # Violeta              
    for sb in (Subjs): 
        for ses in os.listdir(main+sep+sb):
            if 'ses' in ses:
                os.makedirs('topup_'+str(sb)+sep+ses, exist_ok=True)
                for file in os.listdir('.'):
                    if fnmatch.fnmatch(file, '*'+str(sb)+str(ses)+'*'):
                        shutil.move(file, 'topup_'+str(sb)+sep+str(ses)+sep+file)
    print('> moved by Violeta.')

###########################
#        functions        #
###########################

import fnmatch
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces import fsl
from nipype.testing import example_data
from nipype.interfaces import fsl
from nipype.testing import example_data
from nipype.interfaces.fsl import Merge
from nipype.interfaces.fsl import TOPUP
from nipype.interfaces.fsl import ApplyTOPUP
from nipype.interfaces import fsl
from nipype.interfaces.fsl import Eddy
import shutil


def create_params(params_files):
    print("> creating params",print_time(),"      ", end="", flush=True)
    json_params(params_files,main,'acq_params-'+sb+ses+'.txt')

def fsl_roi(in_roi,out_roi):
    print("> creating roi",print_time(),"      ", end="", flush=True)
    fslroi = ExtractROI(in_file=in_roi, roi_file=out_roi, t_min=0,t_size=1)
    fslroi.cmdline #== 'fslroi %s nodif.nii 0 1' % dwidata
    fslroi.run()
    
def fsl_flirt(flirt_file,ref_file):
    print("> creating flirt",print_time(),"      ", end="", flush=True)
    flt = fsl.FLIRT(bins=640, cost_func='mutualinfo')
    flt.inputs.in_file = flirt_file
    flt.inputs.reference = ref_file
    flt.inputs.output_type = "NIFTI_GZ"
    flt.cmdline
    #'flirt -in structural.nii -ref mni.nii -out structural_flirt.nii.gz -omat structural_flirt.mat -bins 640 -searchcost mutualinfo'
    res = flt.run()
    # MOVE FLIES!!!!!
 
def fsl_merge(merge_file1, merge_file2):
    print("> creating merge",print_time(),"      ", end="", flush=True)
    merger = Merge()
    merger.inputs.in_files = [merge_file1, merge_file2]
    merger.inputs.dimension = 't'
    merger.inputs.output_type = 'NIFTI_GZ'
    merger.cmdline
    #'fslmerge -t functional2_merged.nii.gz functional2.nii functional3.nii'
    res=merger.run()
    # MOVE FLIES!!!!!
    
def fsl_topup(imain,topup_params):
    print("> creating fieldmap",print_time(),"      ", end="", flush=True)
    topup = TOPUP()
    topup.inputs.in_file = imain
    topup.inputs.encoding_file = topup_params
    topup.inputs.output_type = "NIFTI_GZ"
    topup.cmdline 

def fsl_apply_topup(imain_up,imain_down,apply_params,apply_fieldcoef,apply_movpar):
    print("> applying fieldmap",print_time(),"      ", end="", flush=True)
    applytopup = ApplyTOPUP()
    applytopup.inputs.in_files = [imain_up,imain_down]
    applytopup.inputs.encoding_file = apply_params
    applytopup.inputs.in_topup_fieldcoef = apply_fieldcoef
    applytopup.inputs.in_topup_movpar = apply_movpar
    applytopup.inputs.output_type = "NIFTI_GZ"
    applytopup.cmdline 
    res = applytopup.run()

def fsl_BET(in_BET,out_BET):
    print("> creating mask",print_time(),"      ", end="", flush=True)
    btr = fsl.BET() 
    btr.inputs.in_file = in_BET
    btr.inputs.out_file = out_BET
    btr.inputs.frac = 0.2
    btr.inputs.mask = True
    res = btr.run()

def fsl_Eddy(in_eddy,mask_eddy,index_eddy,params_eddy,bvec_eddy,bval_eddy):
    print("> applying mask...",print_time(),"      ", end="", flush=True)
    eddy = Eddy()
    eddy.inputs.in_file = in_eddy #--imain= File containing all the images to estimate distortions for flag (dwidata)
    eddy.inputs.in_mask  = mask_eddy
    eddy.inputs.in_index = index_eddy
    eddy.inputs.in_acqp  = params_eddy
    eddy.inputs.in_bvec  = bvec_eddy
    eddy.inputs.in_bval  = bval_eddy
    #Run Eddy
    res = eddy.run()

###########################
###########################

check_dir(main)

print('---------------------')

You are looking in: C:\Users\pablo\OneDrive - Universidad Rey Juan Carlos\Biomedical Engineering URJC\Prácticas+Proyectos+Empresas+Contactos\Mario Gil Correa - Practicas-Biomedica\Documents\SENECA-PICASSO\niftii 
Your code is in   : C:\Users\pablo\OneDrive - Universidad Rey Juan Carlos\Biomedical Engineering URJC\Prácticas+Proyectos+Empresas+Contactos\Mario Gil Correa - Practicas-Biomedica\Documents\SENECA-PICASSO\niftii
---------------------


---

In [63]:
Subjs=[]
for subj in os.listdir(main):
    if subj.startswith('sub'):
        #print(subj)#ckpt
        Subjs.append(subj)

proceed_with=[]
for sb in (Subjs):
    for ses in os.listdir(main+sep+sb):
        if 'ses' in ses:
            mario = input('proceed with '+ sb+ses+'? \n --> If so, enter "y". \n --> If not, enter "no". \n')
            if mario=='y': 
                proceed_with.append(sb+ses)
                print(sb,ses,'added. \n -----')
            else:
                print(sb,ses,'not added. \n -----')

for sb in Subjs:
    for ses in os.listdir(main+sep+sb):
        if 'ses' in ses:
            if sb+ses in proceed_with:
                print('----- \n>> ',sb,ses,':')
                
                directory_dwi=main+sep+sb+sep+ses+sep+'dwi'
                directory_fmap=main+sep+sb+sep+ses+sep+'fmap'
                params_files=[directory_dwi+sep+sb+ses+'_dir-AP_dwi.json',directory_fmap+sep+sb+ses+'_acq-dwi_dir-1_epi.json']
                create_params(params_files)
                print("[DONE] ",print_time())

                in_roi=main+sep+sb+sep+ses+sep+'dwi'+sep+sb+ses+'_dir-AP_dwi.nii'
                out_roi=main+sep+sb+sep+ses+sep+'dwi'+sep+'nodif_'+sb+ses+'-dwi.nii.gz'
                fsl_roi(in_roi,out_roi)
                print("[DONE] ",print_time())

                in_roi=main+sep+sb+sep+ses+sep+'fmap'+sep+sb+ses+'_acq-dwi_dir-1_epi.nii'
                out_roi=main+sep+sb+sep+ses+sep+'fmap'+sep+'nodif_'+sb+ses+'-fmap.nii.gz'
                fsl_roi(in_roi,out_roi)
                print("[DONE] ",print_time())

                ref_file =main+sep+sb+sep+ses+sep+'dwi'+sep+'nodif_'+sb+ses+'-dwi.nii.gz'
                flirt_file=main+sep+sb+sep+ses+sep+'fmap'+sep+'nodif_'+sb+ses+'-fmap.nii.gz'
                fsl_flirt(flirt_file,ref_file)
                print("[DONE] ",print_time())

                merge_file1= main+sep+'nodif_'+sb+ses+'-fmap_flirt.nii.gz'
                merge_file2=main+sep+sb+sep+ses+sep+'dwi'+sep+'nodif_'+sb+ses+'-dwi.nii.gz'
                fsl_merge(merge_file1, merge_file2)
                print("[DONE] ",print_time())

                imain=main+sep+'nodif_'+sb+ses+'-fmap_flirt_merged.nii.gz'
                params=main+sep+'acq_params-'+sb+ses+'.txt'
                fsl_topup(imain,topup_params)
                print("[DONE] ",print_time())
                
                imain_up= main+sep+'nodif_'+sb+ses+'-fmap_flirt.nii.gz'
                imain_down=main+sep+sb+sep+ses+sep+'dwi'+sep+'nodif_'+sb+ses+'-dwi.nii.gz'
                apply_params=main+sep+'acq_params-'+sb+ses+'.txt'
                apply_movpar=main+sep+'nodif_'+sb+ses+'-fmap_flirt_merged_base_movpar.txt'
                apply_fieldcoef=main+sep+'nodif_'+sb+ses+'-fmap_flirt_merged_base_fieldcoef.nii.gz'
                fsl_apply_topup(imain_up,imain_down,apply_params,apply_fieldcoef,apply_movpar)
                print("[DONE] ",print_time())

                in_BET='nodif_'+sb+ses+'-fmap_flirt_corrected.nii.gz'
                out_BET='input_brain.nii'
                fsl_BET(in_BET,out_BET)
                print("[DONE] ",print_time())
                
                file= main+sep+sb+sep+ses+sep+'dwi'+sep+sb+ses+'_dir-AP_dwi.bval'
                save_dir=main
                name='index-'+sb+ses+'_dir-AP_dwi.txt'
                create_index(file,save_dir,name)
                print("[DONE] ",print_time())
                
                in_eddy=main+sep+sb+sep+ses+sep+'dwi'+sep+sb+ses+'_dir-AP_dwi.nii'
                mask_eddy=main+'/input_brain_mask.nii.gz'
                index_eddy=main+sep+'index-'+sb+ses+'_dir-AP_dwi.txt'
                params_eddy=main+'/acq_params-'+sb+ses+'.txt'
                bvec_eddy=main+sep+sb+sep+ses+sep+'/dwi'+sep+sb+ses+'_dir-AP_dwi.bvec'
                bval_eddy=main+sep+sb+sep+ses+sep+'dwi'+sep+sb+ses+'_dir-AP_dwi.bval'
                fsl_Eddy(in_eddy,mask_eddy,index_eddy,params_eddy,bvec_eddy,bval_eddy)
                print("[DONE] ",print_time())
    #tidy_up()
            else:
                print('\n ----- \nskiping',sb,ses,'... \n -----')

proceed with sub-sp01ses-01? 
 --> If so, enter "y". 
 --> If not, enter "no". 
n
sub-sp01 ses-01 not added. 
 -----
proceed with sub-sp01ses-02? 
 --> If so, enter "y". 
 --> If not, enter "no". 
y
sub-sp01 ses-02 added. 
 -----
proceed with sub-sp02ses-01? 
 --> If so, enter "y". 
 --> If not, enter "no". 
n
sub-sp02 ses-01 not added. 
 -----
proceed with sub-sp02ses-02? 
 --> If so, enter "y". 
 --> If not, enter "no". 
y
sub-sp02 ses-02 added. 
 -----

 ----- 
skiping sub-sp01 ses-01 ... 
 -----
----- 
>>  sub-sp01 ses-02 :
[DONE]  17:41:20
[DONE]  17:41:20
[DONE]  17:41:20
[DONE]  17:41:20
[DONE]  17:41:20
[DONE]  17:41:20
[DONE]  17:41:20
[DONE]  17:41:20
> creating index 17:41:20       [DONE]  17:41:20
[DONE]  17:41:20

 ----- 
skiping sub-sp02 ses-01 ... 
 -----
----- 
>>  sub-sp02 ses-02 :
[DONE]  17:41:20
[DONE]  17:41:20
[DONE]  17:41:20
[DONE]  17:41:20
[DONE]  17:41:20
[DONE]  17:41:20
[DONE]  17:41:20
[DONE]  17:41:20
> creating index 17:41:20       [DONE]  17:41:20
[DONE

---
### Trying function on their own...:

In [ ]:
Subjs=[]
for subj in os.listdir(main):
    if subj.startswith('sub'):
        #print(subj)#ckpt
        Subjs.append(subj)

proceed_with=[]
for sb in (Subjs):
    for ses in os.listdir(main+sep+sb):
        if ('ses' in ses) and ('sub' not in ses):
            mario = input('\nproceed with '+ sb+ses+'? \n --> If so, enter "y". \n --> If not, enter "no". \n')
            if mario=='y': 
                proceed_with.append(sb+ses)
                print('',sb,ses,'added.')
                print('-------')
                
for sb in Subjs:
    for ses in os.listdir(main+sep+sb):
        if ('ses' in ses) and not ('sub' in ses):
            if sb+ses in proceed_with:
                print('\n----- RUNNING ALONE -----')
                print('\n>> ',sb,ses,':')
                
                ## INSERT HERE FUNCTION TO RUN ALONE
                file= main+sep+sb+sep+ses+sep+'\dwi'+sep+sb+ses+'_dir-AP_dwi.bval'
                save_dir=main
                name=sb+ses+'_dir-AP_dwi-index.txt'
                create_index(file,save_dir,name)

In [15]:
eddy = Eddy()

#--imain= File containing all the images to estimate distortions for flag (dwidata)
eddy.inputs.in_file = main+'/sub-sp01/ses-02/dwi/sub-sp01ses-02_dir-AP_dwi.nii'

#--mask= File containing the brain mask corresponding to flag
eddy.inputs.in_mask  = main+'/input_brain_mask.nii.gz'

#--index= file containing the indices for all volumes in --imain into --acqp and --topup
eddy.inputs.in_index = main+'/index.txt'

#--acqp= File containing the acquisition parameters
eddy.inputs.in_acqp  = main+'/acq_params-sub-sp01ses-02.txt'

#--bvecs= File containing the b-vectors for all volumes in --imain
eddy.inputs.in_bvec  = main+'/sub-sp01/ses-02/dwi/sub-sp01ses-02_dir-AP_dwi.bvec'

#--bvals= File containing the b-values for all volumes in --imain
eddy.inputs.in_bval  = main+'/sub-sp01/ses-02/dwi/sub-sp01ses-02_dir-AP_dwi.bval'

#Run Eddy
res = eddy.run()

---

<div class="alert alert-block alert-warning">
    Contact me:</b> <br> 
    <b>$$\rightarrow Pablo \quad Laso\quad Mielgo\quad :)$$<br> $$\rightarrow p.laso.2017@alumnos.urjc.es $$
</div>